In [15]:
import pandas as pd
import numpy as np
import ast
pd.set_option('display.max_rows', None)

In [16]:
df = pd.read_csv(r"../output/debug_newmodels.csv")

In [17]:
# List of metric names
metric_names = ["f1","mse","accuracy","recall","precision", "auc","area_under_pr", "r2_score", "rmse"]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset['output_metrics'] = dataset['output_metrics'].apply(ast.literal_eval)
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset['output_metrics'].apply(lambda x: x.get(metric) if isinstance(x, dict) else np.nan)

In [18]:
# Specify the metric you want to maximize
metric_to_maximize = 'mse'
tomax = {"f1": True,
         "mse": False,
         "accuracy": True,
         "recall": True,
         "precision": True,
         "auc": True,
         "area_under_pr": True,
        "rmse": False,
        "r2_score": True}


# Filter out rows with NaN values in the specified metric column
filtered_df = dataset.dropna(subset=[metric_to_maximize]).loc[dataset[metric_to_maximize] != "nan"]

# Find the rows that maximize the specified metric for each dataset
if  tomax[metric_to_maximize]:
    max_rows = filtered_df.sort_values(["dataset",metric_to_maximize])
else:
    max_rows = filtered_df.sort_values(metric_to_maximize)
    
sel_cols = ['dataset', 'model', 
       'eval_metric', 'best_params', 'output_metrics',
 'debug_ytrue', 'f1', 'mse', 'accuracy',
       'recall', 'precision', 'auc', 'area_under_pr', 'r2_score', 'rmse']

max_rows[sel_cols].head(50)

,dataset,model,eval_metric,best_params,output_metrics,debug_ytrue,f1,mse,accuracy,recall,precision,auc,area_under_pr,r2_score,rmse


In [8]:
# Specify the metric you want to maximize
metric_to_maximize = 'f1'
tomax = {"f1": True,
         "mse": False,
         "accuracy": True,
         "recall": True,
         "precision": True,
         "auc": True,
         "area_under_pr": True,
        "rmse": False,
        "r2_score": True}



# Filter out rows with NaN values in the specified metric column
filtered_df = dataset.dropna(subset=[metric_to_maximize]).loc[dataset[metric_to_maximize] != "nan"]

# Find the rows that maximize the specified metric for each dataset
if  tomax[metric_to_maximize]:
    max_rows = filtered_df.groupby('dataset')[metric_to_maximize].idxmax()
else:
    max_rows = filtered_df.groupby('dataset')[metric_to_maximize].idxmin()
    
# Retrieve the rows that maximize the metric
maximized_rows = filtered_df.loc[max_rows]

maximized_rows

,run_id,run_config,dataset,model,execution_mode,eval_metric,best_params,output_metrics,saved_model_path,run_time,...,debug_ytrue,f1,mse,accuracy,recall,precision,auc,area_under_pr,r2_score,rmse
17,202307-2922-4557-aeef7ab7-4348-4aaa-98d5-97124...,"{'dataset': 'breastcancer', 'model': 'fttransf...",breastcancer,fttransformer,hyperopt,roc_auc,"{'Adam_lr': 0.0003065503518754533, 'Adam_weigh...","{'recall': 0.9777777777777777, 'precision': 0....",./output/modelsaves/breastcancer/fttransformer...,107.964507,...,"[1, 0, 1, 1, 0, 1, 1, 1, 1, 0]",0.977778,None,0.972028,0.977778,0.977778,0.998532,0.970035,None,None
0,202307-2920-4202-2d0ceba3-3cc1-49a0-b504-cea64...,"{'dataset': 'iris', 'model': 'xgb', 'best_para...",iris,xgb,hyperopt,accuracy,"{'alpha': 0.7605962861221638, 'colsample_bytre...","{'accuracy': 0.9736842105263158, 'f1': 0.97368...",./output/modelsaves/iris/xgb/202307-2920-4202-...,1.348606,...,"[2, 0, 2, 1, 2, 0, 0, 0, 0, 1]",0.973684,None,0.973684,NaN,NaN,NaN,NaN,None,None
11,202307-2921-1224-13f12be4-9f63-40bf-a0c1-04ce3...,"{'dataset': 'titanic', 'model': 'gate', 'best_...",titanic,gate,hyperopt,accuracy,"{'Adam_lr': 0.001, 'Adam_weight_decay': 0.0001...","{'recall': 0.7093023255813954, 'precision': 0....",./output/modelsaves/titanic/gate/202307-2921-1...,4265.136805,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0.753086,None,0.820628,0.709302,0.802632,0.851638,0.681416,None,None


In [9]:
maximized_rows["best_params"].iloc[0]

"{'Adam_lr': 0.0003065503518754533, 'Adam_weight_decay': 0.0018527915294425184, 'ReduceLROnPlateau_factor': 0.15703250402891672, 'ReduceLROnPlateau_patience': 8, 'add_norm_dropout': 0.09255893395156861, 'attn_dropout': 0.11435052748496637, 'attn_feature_importance': False, 'batch_norm_continuous_input': False, 'batch_size': 128, 'early_stopping_patience': 6, 'embedding_bias': False, 'embedding_dropout': 0.10273940427370434, 'embedding_initialization': 'kaiming_uniform', 'ff_dropout': 0.13034522714427235, 'ff_hidden_multiplier': 4, 'input_embed_dim': 53, 'learning_rate': 0.0013483741168452425, 'num_attn_blocks': 7, 'num_heads': 9, 'optimizer_fn': <class 'torch.optim.adam.Adam'>, 'scheduler_fn': 'ReduceLROnPlateau', 'share_embedding': True, 'share_embedding_strategy': 'add', 'shared_embedding_fraction': 0.32653028909847637, 'transformer_activation': 'ReLU', 'transformer_head_dim': 72, 'outer_params': {'max_epochs': 300, 'early_stopping_tolerance': 1e-06, 'hyperopt_evals': 10, 'val_size':

In [17]:
maximized_rows["best_params"].iloc[1]

"{'alpha': 1, 'colsample_bytree': 0.7701468147287692, 'gamma': 4, 'lambda': 5, 'learning_rate': 0.019628488779340834, 'max_bin': 156, 'max_depth': 8, 'min_child_weight': 8, 'n_estimators': 255, 'subsample': 0.9280869759290384, 'tree_method': 'auto'}"